In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

### Test Data
We will be using the Ordnance Survey Open Roads dataset converted into a graph.  
A sample is shown below:

![Road_Graph](./img/road_graph.png)

This is a weighted graph, using the edge distance in metres.  
__Note__: The Open Roads dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  
*Contains OS data © Crown copyright and database right (2019)*  
Licensed under [Open Government Licence (OGL) V3](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/)

### Read the source data and adjust the vertex IDs

In [2]:
# Test file  - Read OS Open Roads as a graph. (Store dataset in ./data folder)
datafile='./data/gb_road_graph_20190523.csv'

In [3]:
# Read the data file with lenght of edge in metres as weight
cols = ["src", "dst","length"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32"),
        ("length", "float64"),
        ])

gdf = cudf.read_csv(datafile, names=cols, dtype=list(dtypes.values()) ,skiprows=1)

In [4]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [5]:
# Displat the results
print(gdf)

    src  dst  length  src_0  dst_0
0   16   20   162.0     15     19
1   20   16   162.0     19     15
2   56   55   855.0     55     54
3   55   56   855.0     54     55
4   72   22   382.0     71     21
5   22   72   382.0     21     71
6   72   71   270.0     71     70
7   71   72   270.0     70     71
8   17   18    48.0     16     17
9   18   17    48.0     17     16
[7347796 more rows]


### Create the Road Graph and call SSSP

In [6]:
%%time
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"],gdf['length'])

CPU times: user 167 µs, sys: 152 µs, total: 319 µs
Wall time: 326 µs


In [7]:
%%time
# Call cugraph.sssp to get the distances from vertex 1103168 (my house):
df = cugraph.sssp(G,1103168)

CPU times: user 1.87 s, sys: 756 ms, total: 2.63 s
Wall time: 2.62 s


In [8]:
print(df)

    vertex                 distance
0       0  1.7976931348623157e+308
1       1  1.7976931348623157e+308
2       2  1.7976931348623157e+308
3       3  1.7976931348623157e+308
4       4  1.7976931348623157e+308
5       5  1.7976931348623157e+308
6       6  1.7976931348623157e+308
7       7  1.7976931348623157e+308
8       8  1.7976931348623157e+308
9       9  1.7976931348623157e+308
[3078121 more rows]


In [9]:
# Exclude non-connected nodes (islands) set to fp64 max
print(df.query("distance<1.7976931348623157e+308"))

    vertex  distance
8941    8941  730425.0
8942    8942  730387.0
8943    8943  730651.0
8944    8944  730630.0
8945    8945  729755.0
8946    8946  729875.0
8947    8947  729538.0
8948    8948  730048.0
8949    8949  729819.0
8950    8950  729924.0
[3048661 more rows]


### Calculate nearest and farthest vertices

In [10]:
# Get 10 nearest vertices, excluding non-connected
print(df.query("distance<1.7976931348623157e+308").nlargest(10,['distance']))

    vertex  distance
15930   15930  792854.0
15991   15991  791964.0
16253   16253  791838.0
15968   15968  791542.0
15966   15966  791466.0
15971   15971  791391.0
15970   15970  791296.0
15963   15963  791290.0
15969   15969  791214.0
15964   15964  791201.0


In [11]:
# Get 10 farthest vertices, excluding non-connected
print(df.query("distance<1.7976931348623157e+308").nsmallest(10,['distance']))

     vertex  distance
1103168  1103168       0.0
1103174  1103174      53.0
1103179  1103179     144.0
1103177  1103177     159.0
1103229  1103229     173.0
1103181  1103181     196.0
1103184  1103184     216.0
1103185  1103185     231.0
1103186  1103186     236.0
1103187  1103187     330.0


___
Copyright (c) 2019, NVIDIA CORPORATION.

Adapted by John Murray

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___